# FINM 33150: Quantitative Trading Strategies Homework 3, Financial Ratio Quantile Strategy
#### Author: Joshua Weekes
#### Due Date: 2/1/2024

#### Sections:

1. Introduction

2. Package Imports

3. Helper Functions

4. Data Import

5. Trading Helper Functions

6. Parameter Initialization

7. Trading Algorithm Logic

8. Running Trading Algortihm on DIfferent Parameter Sets

9. Results

10. Analysis

11. Conclusion

# Introduction

# Package Imports and API/Date Range Initialization

In [141]:
import os
import pandas as pd
import numpy as np
import quandl
from matplotlib import pyplot as plt
from matplotlib import dates
import seaborn as sns
import statsmodels.api as sm
from datetime import date
import datetime
import functools
import random

from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay

us_business_day = CustomBusinessDay(calendar=USFederalHolidayCalendar())

global api_key
api_key = 'ykKjRrdMDZk8sVyrhPEV'
start_date_str = '2016-01-01'
end_date_str = '2023-06-30'
start_date = datetime.datetime.strptime(start_date_str, '%Y-%m-%d')
end_date = datetime.datetime.strptime(end_date_str, '%Y-%m-%d')

# Helper Functions

In [2]:
def grab_quandl_table(
    table_path,
    avoid_download=False,
    replace_existing=False,
    date_override=None,
    allow_old_file=False,
    **kwargs,
):
    root_data_dir = os.path.join(r'C:\Users\joshw\OneDrive\Desktop\UChicago\FINM_33150_quantitative_trading_strategies\data', 'quandl_data_table_downloads')
    data_symlink = os.path.join(root_data_dir, f"{table_path}.zip")
    if avoid_download and os.path.exists(data_symlink):
        print(f"Skipping any possible download of {table_path}")
        return data_symlink
    
    table_dir = os.path.dirname(data_symlink)
    if not os.path.isdir(table_dir):
        print(f'Creating new data dir {table_dir}')
        os.mkdir(table_dir)

    if date_override is None:
        my_date = datetime.datetime.now().strftime("%Y%m%d")
    else:
        my_date = date_override
    data_file = os.path.join(root_data_dir, f"{table_path}_{my_date}.zip")

    if os.path.exists(data_file):
        file_size = os.stat(data_file).st_size
        if replace_existing or not file_size > 0:
            print(f"Removing old file {data_file} size {file_size}")
        else:
            print(
                f"Data file {data_file} size {file_size} exists already, no need to download"
            )
            return data_file

    dl = quandl.export_table(
        table_path, filename=data_file, api_key=api_key, **kwargs
    )
    file_size = os.stat(data_file).st_size
    if os.path.exists(data_file) and file_size > 0:
        print(f"Download finished: {file_size} bytes")
        if not date_override:
            if os.path.exists(data_symlink):
                print(f"Removing old symlink")
                os.unlink(data_symlink)
            print(f"Creating symlink: {data_file} -> {data_symlink}")
            os.symlink(
                data_file, data_symlink,
            )
    else:
        print(f"Data file {data_file} failed download")
        return
    return data_symlink if (date_override is None or allow_old_file) else "NoFileAvailable"

def fetch_quandl_table(table_path, avoid_download=True, **kwargs):
    return pd.read_csv(
        grab_quandl_table(table_path, avoid_download=avoid_download, **kwargs)
    )

# Ratio and Description Data Import

In [62]:
ratio_columns = ['m_ticker', 'ticker', 'currency_code', 'per_end_date', 'per_type', 'tot_debt_tot_equity', 'ret_invst']

ratio_df = fetch_quandl_table('ZACKS/FR', avoid_download=True, per_end_date={'gte': start_date, 'lte': end_date}, per_type='Q', qopts={'columns': ratio_columns})
# print(len(ratio_df['m_ticker'].unique().tolist()))
ratio_df = ratio_df.dropna(subset=['m_ticker'])
# print(len(ratio_df['m_ticker'].unique().tolist()))
# ratio_df['per_end_date'] = pd.to_datetime(ratio_df['per_end_date'], format='%Y-%m-%d')
# ratio_df.loc[ratio_df['m_ticker'] == 'A2']

Data file C:\Users\joshw\OneDrive\Desktop\UChicago\FINM_33150_quantitative_trading_strategies\data\quandl_data_table_downloads\ZACKS/FR_20240201.zip size 2048080 exists already, no need to download


In [63]:
type(ratio_df['per_end_date'][0])

str

In [66]:
unique_ticker_ratio_df = ratio_df.m_ticker.unique().tolist()
description_columns = ['m_ticker', 'ticker', 'active_ticker_flag','zacks_x_sector_code']

description_df = fetch_quandl_table('ZACKS/MT', avoid_download=True, m_ticker=unique_ticker_ratio_df, qopts={'columns': description_columns})
# print(len(description_df['m_ticker'].unique().tolist()))
description_df = description_df.dropna()
# print(len(description_df['m_ticker'].unique().tolist()))
description_df = description_df[description_df['active_ticker_flag'] == 'Y']
description_df = description_df[~ description_df['zacks_x_sector_code'].isin([0.0, 5.0, 13])]
print(len(description_df['m_ticker'].unique().tolist()))

# description_df.head()

Data file C:\Users\joshw\OneDrive\Desktop\UChicago\FINM_33150_quantitative_trading_strategies\data\quandl_data_table_downloads\ZACKS/MT_20240201.zip size 68719 exists already, no need to download
6122


In [88]:
fundamental_columns = ['m_ticker', 'ticker', 'per_end_date', 'per_type', 'filing_date', 'eps_diluted_net', 'basic_net_eps', 'net_lterm_debt', 'tot_lterm_debt']
description_unique_ticker = description_df.m_ticker.unique().tolist()

fundamental_df = fetch_quandl_table('ZACKS/FC', avoid_download=True, per_end_date={'gte': start_date_str, 'lte': end_date_str}, per_type='Q', qopts={'columns': fundamental_columns})
# fundamental_df = fundamental_df.dropna()
fundamental_df = fundamental_df[fundamental_df['m_ticker'].isin(description_unique_ticker)]
fundamental_df['net_lterm_debt'] = fundamental_df['net_lterm_debt'].fillna(0)
print(len(fundamental_df['m_ticker'].unique().tolist()))
fundamental_df = fundamental_df.sort_values(by=['m_ticker', 'per_end_date'], ascending=True)
# fundamental_df['per_end_date'] = pd.to_datetime(fundamental_df['per_end_date'], format='%Y-%m-%d')
# fundamental_df.loc[fundamental_df['m_ticker'] == 'A2']

Data file C:\Users\joshw\OneDrive\Desktop\UChicago\FINM_33150_quantitative_trading_strategies\data\quandl_data_table_downloads\ZACKS/FC_20240201.zip size 2706406 exists already, no need to download
6122


In [87]:
shares_out_columns = ['m_ticker', 'ticker', 'per_end_date', 'per_type', 'shares_out']

shares_out_df = fetch_quandl_table('ZACKS/SHRS', avoid_download=True, per_end_date={'gte': start_date_str, 'lte': end_date_str}, qopts={'columns': shares_out_columns})
# shares_out_df = shares_out_df.dropna()
shares_out_df['m_ticker'] = shares_out_df['m_ticker'].str.replace(' ', '')
shares_out_df = shares_out_df[shares_out_df['m_ticker'].isin(description_unique_ticker)]
print(len(shares_out_df['m_ticker'].unique().tolist()))
# shares_out_df.loc[shares_out_df['m_ticker'] == 'A2']
shares_out_unique_ticker = shares_out_df.m_ticker.unique().tolist()
shares_out = shares_out_df.sort_values(by=['m_ticker', 'per_end_date'], ascending=[True, True])
# shares_out_df['per_end_date'] = pd.to_datetime(shares_out_df['per_end_date'], format='%Y-%m-%d')

Data file C:\Users\joshw\OneDrive\Desktop\UChicago\FINM_33150_quantitative_trading_strategies\data\quandl_data_table_downloads\ZACKS/SHRS_20240201.zip size 1684657 exists already, no need to download
5173


In [70]:
mkvt_columns = ['m_ticker', 'ticker', 'per_end_date', 'per_type', 'mkt_val']

mktv_df = fetch_quandl_table('ZACKS/MKTV', avoid_download=True, per_end_date={'gte': start_date_str, 'lte': end_date_str}, qopts={'columns': mkvt_columns})
# mktv_df = mktv_df.dropna()
mktv_df['m_ticker'] = mktv_df['m_ticker'].str.replace(' ', '')
mktv_df = mktv_df[mktv_df['m_ticker'].isin(description_unique_ticker)]
print(len(mktv_df['m_ticker'].unique().tolist()))
# mktv_df['per_end_date'] = pd.to_datetime(mktv_df['per_end_date'], format='%Y-%m-%d')
mktv_df = mktv_df.sort_values(by=['m_ticker', 'per_end_date'], ascending=[True, True])
# mktv_df.loc[mktv_df['m_ticker'] == 'A2']

Data file C:\Users\joshw\OneDrive\Desktop\UChicago\FINM_33150_quantitative_trading_strategies\data\quandl_data_table_downloads\ZACKS/MKTV_20240201.zip size 2058832 exists already, no need to download
5173


In [44]:
ratio_df = ratio_df.loc[ratio_df['m_ticker'].isin(shares_out_unique_ticker)]
fundamental_df = fundamental_df.loc[fundamental_df['m_ticker'].isin(shares_out_unique_ticker)]
print(len(ratio_df['m_ticker'].unique().tolist()))
print(len(fundamental_df['m_ticker'].unique().tolist()))

5173
5173


In [71]:
def valid_tickers(df, unique_tickers):
    # List to hold tickers that meet the criteria
    valid_tickers = []
    valid_tickers_count = 0
    invalid_ticker_len = []
    
    # Iterate over each unique ticker in the 'm_ticker' column
    for ticker in unique_tickers:
        # Filter the DataFrame for the current ticker
        filtered_df = df[df['m_ticker'] == ticker]
        
        # Check if there are exactly 30 unique dates for the current ticker
        if filtered_df['per_end_date'].nunique() == 30:
            valid_tickers.append(ticker)
            valid_tickers_count += 1
        else:
            invalid_ticker_len.append(filtered_df['per_end_date'].nunique())
    
    return valid_tickers, valid_tickers_count, invalid_ticker_len

In [73]:
ratio_valid_tickers, ratio_valid_tickers_count, ratio_invalid_lens = valid_tickers(ratio_df, shares_out_unique_ticker)
fundamental_valid_tickers, fundamental_valid_tickers_count, fundamental_invalid_lens = valid_tickers(fundamental_df, shares_out_unique_ticker)
shares_out_valid_tickers, shares_out_valid_tickers_count, shares_out_invalid_lens = valid_tickers(shares_out_df, shares_out_unique_ticker)
mktv_valid_tickers, mktv_valid_tickers_count, mktv_invalid_lens = valid_tickers(mktv_df, shares_out_unique_ticker)

In [72]:
ratio_valid_tickers_count

2672

In [47]:
print(ratio_valid_tickers_count)
print(fundamental_valid_tickers_count)
print(shares_out_valid_tickers_count)
print(mktv_valid_tickers_count)

2672


In [89]:
ratio_df = ratio_df.loc[ratio_df['m_ticker'].isin(ratio_valid_tickers)]
fundamental_df = fundamental_df.loc[fundamental_df['m_ticker'].isin(ratio_valid_tickers)]
shares_out_df = shares_out_df.loc[shares_out_df['m_ticker'].isin(ratio_valid_tickers)]
mktv_df = mktv_df.loc[mktv_df['m_ticker'].isin(ratio_valid_tickers)]

In [81]:
print(len(ratio_df['m_ticker'].unique().tolist()))
print(len(fundamental_df['m_ticker'].unique().tolist()))
print(len(shares_out_df['m_ticker'].unique().tolist()))
print(len(mktv_df['m_ticker'].unique().tolist()))

print(len(ratio_df))
print(len(fundamental_df))
print(len(shares_out_df))
print(len(mktv_df))

2672
2672
2672
2672
80160
80160
80160
80160


In [86]:
print(set(ratio_df['m_ticker'].unique().tolist()) == set(fundamental_df['m_ticker'].unique().tolist()) == set(shares_out_df['m_ticker'].unique().tolist()) == set(mktv_df['m_ticker'].unique().tolist()))

True


In [49]:
mktv_df.head()

,m_ticker,ticker,per_end_date,per_type,mkt_val
959,#AAO,AAON,2016-03-31,Q,1485.93
958,#AAO,AAON,2016-06-30,Q,1458.66
957,#AAO,AAON,2016-09-30,Q,1525.30
956,#AAO,AAON,2016-12-31,Q,1743.55
955,#AAO,AAON,2017-03-31,Q,1860.47


In [77]:
len(mktv_df['m_ticker'].unique().tolist())

2672

In [90]:
def same_merge_date(merge_df, df):
    # List to hold tickers that meet the criteria
    same_merge_date_count = 0
    same_merge_date_tickers = []
    
    # Iterate over each unique ticker in the 'm_ticker' column
    for ticker in merge_df['m_ticker'].unique().tolist():
        # Filter the DataFrame for the current ticker
        filtered_merge_df = merge_df.loc[merge_df['m_ticker'] == ticker]
        filtered_df = df.loc[df['m_ticker'] == ticker]
        
        # Check if there are exactly 30 unique dates for the current ticker
        if set(filtered_merge_df['per_end_date'].unique().tolist()) == set(filtered_df['per_end_date'].unique().tolist()):
            same_merge_date_count += 1
            same_merge_date_tickers.append(ticker)
    
    return same_merge_date_count, same_merge_date_tickers

In [91]:
merge1_same_merge_date_count, merge1_same_merge_date_tickers = same_merge_date(ratio_df, fundamental_df)
merge1_same_merge_date_count

2672

In [92]:
merge2_same_merge_date_count, merge2_same_merge_date_tickers = same_merge_date(ratio_df, shares_out_df)
merge2_same_merge_date_count

2672

In [93]:
merge3_same_merge_date_count, merge3_same_merge_date_tickers = same_merge_date(ratio_df, mktv_df)
merge3_same_merge_date_count

2395

In [94]:
ratio_df = ratio_df.loc[ratio_df['m_ticker'].isin(merge3_same_merge_date_tickers)]
fundamental_df = fundamental_df.loc[fundamental_df['m_ticker'].isin(merge3_same_merge_date_tickers)]
shares_out_df = shares_out_df.loc[shares_out_df['m_ticker'].isin(merge3_same_merge_date_tickers)]
mktv_df = mktv_df.loc[mktv_df['m_ticker'].isin(merge3_same_merge_date_tickers)]

print(len(ratio_df['m_ticker'].unique().tolist()))
print(len(fundamental_df['m_ticker'].unique().tolist()))
print(len(shares_out_df['m_ticker'].unique().tolist()))
print(len(mktv_df['m_ticker'].unique().tolist()))

2395
2395
2395
2395


In [202]:
zacks_df = pd.merge(fundamental_df, ratio_df[['m_ticker', 'per_end_date', 'currency_code', 'tot_debt_tot_equity', 'ret_invst']], on=['m_ticker', 'per_end_date'], how='left')
zacks_df = pd.merge(zacks_df, shares_out_df[['m_ticker', 'per_end_date', 'shares_out']], on=['m_ticker', 'per_end_date'], how='left')
zacks_df = pd.merge(zacks_df, mktv_df[['m_ticker', 'per_end_date', 'mkt_val']], on=['m_ticker', 'per_end_date'], how='left')

zacks_df['tot_debt_tot_equity'] = zacks_df['tot_debt_tot_equity'].fillna(0)
zacks_df['tot_lterm_debt'] = zacks_df['tot_lterm_debt'].fillna(0)

zacks_df['filing_date'] = pd.to_datetime(zacks_df['filing_date'], format='%Y-%m-%d')
zacks_df['per_end_date'] = pd.to_datetime(zacks_df['per_end_date'], format='%Y-%m-%d')

zacks_df['filing_date'] = zacks_df['filing_date'] + us_business_day

zacks_df = zacks_df.drop(['per_type'], axis=1)

# zacks_df['per_end_date'] = pd.to_datetime(zacks_df['per_end_date'], format='%Y-%m-%d')

C:\Users\joshw\AppData\Local\Temp\ipykernel_23172\1801510477.py:11: PerformanceWarning: Non-vectorized DateOffset being applied to Series or DatetimeIndex.
  zacks_df['filing_date'] = zacks_df['filing_date'] + us_business_day


In [203]:
def tot_debt_tot_equity_check_tickers(df):
    # List to hold tickers that meet the criteria
    tot_debt_tot_equity_check_m_tickers = []
    tot_debt_tot_equity_check_tickers = []
    ticker_dict = {}
    
    # Iterate over each unique ticker in the 'm_ticker' column
    for m_ticker in df['m_ticker'].unique().tolist():
        # Filter the DataFrame for the current ticker
        filtered_df = df[df['m_ticker'] == m_ticker]
        
        # Check if there are exactly 30 unique dates for the current ticker
        if (filtered_df['tot_debt_tot_equity'] > .1).sum() > (filtered_df['tot_debt_tot_equity'] < .1).sum():
            tot_debt_tot_equity_check_m_tickers.append(m_ticker)
            tot_debt_tot_equity_check_tickers.append(filtered_df['ticker'].unique().tolist()[-1])
            ticker_dict[m_ticker] = filtered_df['ticker'].unique().tolist()[-1]

    
    return tot_debt_tot_equity_check_m_tickers, tot_debt_tot_equity_check_tickers, ticker_dict

In [204]:
zacks_check_m_tickers, zacks_check_tickers, ticker_dict = tot_debt_tot_equity_check_tickers(zacks_df)

In [205]:
print(len(zacks_check_m_tickers))
print(len(zacks_check_tickers))

1506
1506


In [206]:
print(len(zacks_df['m_ticker'].unique().tolist()))
print(len(zacks_df['ticker'].unique().tolist()))
zacks_df.head(30)

2395
2395


,m_ticker,ticker,per_end_date,filing_date,eps_diluted_net,basic_net_eps,net_lterm_debt,tot_lterm_debt,currency_code,tot_debt_tot_equity,ret_invst,shares_out,mkt_val
0,#AAO,AAON,2016-03-31,2016-05-06,0.1467,0.1333,0.000,0.000,USD,0.0000,6.1223,79.60,1485.93
1,#AAO,AAON,2016-06-30,2016-08-05,0.1800,0.1800,0.000,0.000,USD,0.0000,7.5519,79.54,1458.66
2,#AAO,AAON,2016-09-30,2016-11-04,0.1933,0.2000,0.000,0.000,USD,0.0000,7.6581,79.39,1525.30
3,#AAO,AAON,2016-12-31,2017-02-24,0.1467,0.1467,0.000,0.000,USD,0.0000,5.5455,79.13,1743.55
4,#AAO,AAON,2017-03-31,2017-05-05,0.1267,0.1267,0.000,0.000,USD,0.0000,4.8240,78.94,1860.47
5,#AAO,AAON,2017-06-30,2017-08-04,0.1733,0.1733,0.000,0.000,USD,0.0000,6.3349,78.94,1939.42
6,#AAO,AAON,2017-09-30,2017-11-03,0.1867,0.1867,0.000,0.000,USD,0.0000,6.3904,78.90,1813.51
7,#AAO,AAON,2017-12-31,2018-02-28,0.1867,0.2000,0.000,0.000,USD,0.0000,6.3661,78.70,1925.59
8,#AAO,AAON,2018-03-31,2018-05-04,0.0533,0.0533,0.000,0.000,USD,0.0000,1.7779,78.64,2044.77
9,#AAO,AAON,2018-06-30,2018-08-03,0.1467,0.1467,0.000,0.000,USD,0.0000,4.9032,78.54,1741.04


In [243]:
daily_close_df = fetch_quandl_table('QUOTEMEDIA/PRICES', avoid_download=True, ticker=zacks_check_tickers, date={'gte': start_date_str, 'lte': end_date_str})
daily_close_df['date'] = pd.to_datetime(daily_close_df['date'], format='%Y-%m-%d')
daily_close_df.head(30)

Data file C:\Users\joshw\OneDrive\Desktop\UChicago\FINM_33150_quantitative_trading_strategies\data\quandl_data_table_downloads\QUOTEMEDIA/PRICES_20240201.zip size 92177394 exists already, no need to download


,ticker,date,open,high,low,close,volume,dividend,split,adj_open,adj_high,adj_low,adj_close,adj_volume
0,AA,2016-11-01,22.10,23.5500,21.7800,23.00,32216510.0,0.0,1.0,21.640207,23.060040,21.326865,22.521483,32216510.0
1,AA,2016-11-02,22.67,23.2650,22.6358,22.91,10755328.0,0.0,1.0,22.198349,22.780970,22.164860,22.433355,10755328.0
2,AA,2016-11-03,22.96,24.3493,22.9400,24.15,8083097.0,0.0,1.0,22.482315,23.842711,22.462731,23.647557,8083097.0
3,AA,2016-11-04,23.82,25.5000,23.5600,25.20,6878500.0,0.0,1.0,23.324423,24.969470,23.069832,24.675712,6878500.0
4,AA,2016-11-07,25.54,26.4100,24.8900,25.08,8377025.0,0.0,1.0,25.008638,25.860538,24.372161,24.558208,8377025.0
5,AA,2016-11-08,24.30,26.5900,24.3000,26.40,6469979.0,0.0,1.0,23.794436,26.036793,23.794436,25.850746,6469979.0
6,AA,2016-11-09,26.83,29.1400,26.0000,28.72,10675856.0,0.0,1.0,26.271799,28.533740,25.459068,28.122478,10675856.0
7,AA,2016-11-10,29.08,29.7000,28.3000,29.10,8189887.0,0.0,1.0,28.474988,29.082089,27.711216,28.494572,8189887.0
8,AA,2016-11-11,29.18,29.9900,27.6300,29.30,5337239.0,0.0,1.0,28.572907,29.366055,27.055155,28.690411,5337239.0
9,AA,2016-11-14,29.50,29.9400,28.6100,29.62,5156737.0,0.0,1.0,28.886250,29.317096,28.014766,29.003753,5156737.0


In [239]:
daily_close_df.columns

Index(['ticker', 'date', 'open', 'high', 'low', 'close', 'volume', 'dividend',
       'split', 'adj_open', 'adj_high', 'adj_low', 'adj_close', 'adj_volume'],
      dtype='object')

In [245]:
daily_df = pd.merge(zacks_df, daily_close_df[['ticker', 'date', 'adj_close']], left_on=['ticker', 'filing_date'], right_on=['ticker', 'date'], how='right')
daily_df.head(30)

MemoryError: Unable to allocate 20.1 MiB for an array with shape (2629579,) and data type int64

In [207]:
date_range = pd.date_range(start=start_date, end=end_date, freq=us_business_day)
zacks_tickers = zacks_df['m_ticker'].unique().tolist()

In [208]:
# Create a multi-index from all combinations of stocks and the business day date range
multi_index = pd.MultiIndex.from_product([zacks_tickers, date_range], names=['m_ticker', 'daily_date'])

# Create an expanded DataFrame with this multi-index
empty_daily_df = pd.DataFrame(index=multi_index).reset_index()

# Merge the expanded DataFrame with the original data
zacks_daily_df = pd.merge(empty_daily_df, zacks_df, left_on=['m_ticker', 'daily_date'], right_on=['m_ticker', 'filing_date'], how='left')

# Forward fill the missing quarterly data
forward_fill_columns = ['ticker', 'currency_code', 'filing_date', 'per_end_date', 'tot_debt_tot_equity', 'net_lterm_debt', 'tot_lterm_debt', 'shares_out', 'eps_diluted_net', 'basic_net_eps', 'ret_invst']
zacks_daily_df[forward_fill_columns] = zacks_daily_df.groupby('m_ticker')[forward_fill_columns].ffill()
# zacks_daily_df['currency_code'] = zacks_daily_df.groupby('m_ticker')['currency_code'].ffill()
# zacks_daily_df['filing_date'] = zacks_daily_df.groupby('m_ticker')['filing_date'].ffill()
# zacks_daily_df['per-end_date'] = zacks_daily_df.groupby('m_ticker')['per_end_date'].ffill()
# zacks_daily_df['tot_debt_tot_equity'] = zacks_daily_df.groupby('m_ticker')['tot_debt_tot_equity'].ffill()

first_dates = zacks_daily_df.groupby('m_ticker')['filing_date'].min().rename('FirstDate')

# Merge the first dates back into the original DataFrame
zacks_daily_df = zacks_daily_df.merge(first_dates, on='m_ticker')

# Filter out rows where 'Other_Date' is less than the first non-NaN 'Date'
zacks_daily_df = zacks_daily_df[zacks_daily_df['daily_date'] >= zacks_daily_df['FirstDate']]

# Optionally drop the auxiliary 'FirstDate' column
zacks_daily_df.drop(columns=['FirstDate'], inplace=True)


zacks_daily_df.head(30)
# Drop unnecessary columns and adjust as needed
# final_df = merged_df.drop(columns=['ReportDate']).sort_values(by=['Stock', 'Date'])

,m_ticker,daily_date,ticker,per_end_date,filing_date,eps_diluted_net,basic_net_eps,net_lterm_debt,tot_lterm_debt,currency_code,tot_debt_tot_equity,ret_invst,shares_out,mkt_val
87,#AAO,2016-05-06,AAON,2016-03-31,2016-05-06,0.1467,0.1333,0.0,0.0,USD,0.0,6.1223,79.6,1485.93
88,#AAO,2016-05-09,AAON,2016-03-31,2016-05-06,0.1467,0.1333,0.0,0.0,USD,0.0,6.1223,79.6,NaN
89,#AAO,2016-05-10,AAON,2016-03-31,2016-05-06,0.1467,0.1333,0.0,0.0,USD,0.0,6.1223,79.6,NaN
90,#AAO,2016-05-11,AAON,2016-03-31,2016-05-06,0.1467,0.1333,0.0,0.0,USD,0.0,6.1223,79.6,NaN
91,#AAO,2016-05-12,AAON,2016-03-31,2016-05-06,0.1467,0.1333,0.0,0.0,USD,0.0,6.1223,79.6,NaN
92,#AAO,2016-05-13,AAON,2016-03-31,2016-05-06,0.1467,0.1333,0.0,0.0,USD,0.0,6.1223,79.6,NaN
93,#AAO,2016-05-16,AAON,2016-03-31,2016-05-06,0.1467,0.1333,0.0,0.0,USD,0.0,6.1223,79.6,NaN
94,#AAO,2016-05-17,AAON,2016-03-31,2016-05-06,0.1467,0.1333,0.0,0.0,USD,0.0,6.1223,79.6,NaN
95,#AAO,2016-05-18,AAON,2016-03-31,2016-05-06,0.1467,0.1333,0.0,0.0,USD,0.0,6.1223,79.6,NaN
96,#AAO,2016-05-19,AAON,2016-03-31,2016-05-06,0.1467,0.1333,0.0,0.0,USD,0.0,6.1223,79.6,NaN


In [223]:
len(zacks_check_tickers)

1506

In [236]:
type(zacks_daily_df['filing_date'][87])

pandas._libs.tslibs.timestamps.Timestamp

In [226]:
daily_close_df = fetch_quandl_table('QUOTEMEDIA/PRICES', avoid_download=True, ticker=zacks_check_tickers, date={'gte': start_date_str, 'lte': end_date_str})
daily_close_df['date'] = pd.to_datetime(daily_close_df['date'], format='%Y-%m-%d')
daily_close_df.head(30)

Data file C:\Users\joshw\OneDrive\Desktop\UChicago\FINM_33150_quantitative_trading_strategies\data\quandl_data_table_downloads\QUOTEMEDIA/PRICES_20240201.zip size 92177394 exists already, no need to download


,ticker,date,open,high,low,close,volume,dividend,split,adj_open,adj_high,adj_low,adj_close,adj_volume
0,AA,2016-11-01,22.10,23.5500,21.7800,23.00,32216510.0,0.0,1.0,21.640207,23.060040,21.326865,22.521483,32216510.0
1,AA,2016-11-02,22.67,23.2650,22.6358,22.91,10755328.0,0.0,1.0,22.198349,22.780970,22.164860,22.433355,10755328.0
2,AA,2016-11-03,22.96,24.3493,22.9400,24.15,8083097.0,0.0,1.0,22.482315,23.842711,22.462731,23.647557,8083097.0
3,AA,2016-11-04,23.82,25.5000,23.5600,25.20,6878500.0,0.0,1.0,23.324423,24.969470,23.069832,24.675712,6878500.0
4,AA,2016-11-07,25.54,26.4100,24.8900,25.08,8377025.0,0.0,1.0,25.008638,25.860538,24.372161,24.558208,8377025.0
5,AA,2016-11-08,24.30,26.5900,24.3000,26.40,6469979.0,0.0,1.0,23.794436,26.036793,23.794436,25.850746,6469979.0
6,AA,2016-11-09,26.83,29.1400,26.0000,28.72,10675856.0,0.0,1.0,26.271799,28.533740,25.459068,28.122478,10675856.0
7,AA,2016-11-10,29.08,29.7000,28.3000,29.10,8189887.0,0.0,1.0,28.474988,29.082089,27.711216,28.494572,8189887.0
8,AA,2016-11-11,29.18,29.9900,27.6300,29.30,5337239.0,0.0,1.0,28.572907,29.366055,27.055155,28.690411,5337239.0
9,AA,2016-11-14,29.50,29.9400,28.6100,29.62,5156737.0,0.0,1.0,28.886250,29.317096,28.014766,29.003753,5156737.0


In [235]:
type(daily_close_df['date'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [229]:
# Step 1: Find matching tickers
matching_tickers = set(daily_close_df['ticker']).intersection(set(zacks_daily_df['ticker']))

# Step 2: For matching tickers, check if all old_df dates are in new_df dates for each ticker
tickers_with_all_dates_matched = []

for ticker in matching_tickers:
    # Filter each DataFrame by the current ticker
    daily_close_dates = set(daily_close_df[daily_close_df['ticker'] == ticker]['date'])
    zacks_daily_dates = set(zacks_daily_df[zacks_daily_df['ticker'] == ticker]['daily_date'])
    
    # Check if all old_dates are in new_dates
    if zacks_daily_dates.issubset(daily_close_dates):
        tickers_with_all_dates_matched.append(ticker)

In [230]:
len(tickers_with_all_dates_matched)

0

In [231]:
len(matching_tickers)

1278

In [228]:
len(zacks_daily_df['ticker'].unique().tolist())

2102

In [227]:
len(daily_close_df['ticker'].unique().tolist())

1432

In [ ]:
random.seed(99)

trading_universe = ['LLY']
tickers_with_all_dates_matched = tickers_with_all_dates_matched.remove('LLY')

trading_universe += random.sample(tickers_with_all_dates_matched, 199)
print(trading_universe)

In [ ]:
zacks_df = zacks_df.loc[zacks_df['m_ticker'].isin(trading_universe)]

In [183]:
print(len(daily_close_df['ticker'].unique().tolist()) == len(trading_universe))
print(len(daily_close_df['ticker']) == len(zacks_daily_df['m_ticker']))
print(len(daily_close_df['ticker'].unique().tolist()))


False
False
186


In [54]:
merge2_same_merge_date_count, merge2_same_merge_date_tickers = same_merge_date(zacks_df, shares_out_df)

In [55]:
merge2_same_merge_date_count

0

In [ ]:
zacks_df = pd.merge(zacks_df, shares_out_df[['m_ticker', 'per_end_date', 'shares_out']], on=['m_ticker', 'per_end_date'], how='left')

In [15]:
same_merge_date_count, same_merge_date_tickers = same_merge_date(zacks_df, mktv_df)

In [31]:
zacks_df = zacks_df.loc[zacks_df['m_ticker'].isin(same_merge_date_tickers)]
mktv_df = mktv_df.loc[mktv_df['m_ticker'].isin(same_merge_date_tickers)]
zacks_df = pd.merge(zacks_df, mktv_df[['m_ticker', 'per_end_date', 'mkt_val']], on=['m_ticker', 'per_end_date'], how='left')
print(len(zacks_df['m_ticker'].unique().tolist()))
print(len(same_merge_date_tickers))
zacks_df = zacks_df.sort_values(by=['m_ticker', 'per_end_date'], ascending=[True, True])

zacks_df.head(30)

1555
1555


,m_ticker,ticker,per_end_date,per_type,filing_date,eps_diluted_net,basic_net_eps,net_lterm_debt,tot_lterm_debt,currency_code,tot_debt_tot_equity,ret_invst,shares_out,mkt_val_x,mkt_val_y
0,#AAO,AAON,2016-03-31,Q,2016-05-05,0.1467,0.1333,0.000,NaN,USD,NaN,6.1223,79.60,1485.93,1485.93
1,#AAO,AAON,2016-06-30,Q,2016-08-04,0.1800,0.1800,0.000,NaN,USD,NaN,7.5519,79.54,1458.66,1458.66
2,#AAO,AAON,2016-09-30,Q,2016-11-03,0.1933,0.2000,0.000,NaN,USD,NaN,7.6581,79.39,1525.30,1525.30
3,#AAO,AAON,2016-12-31,Q,2017-02-23,0.1467,0.1467,0.000,NaN,USD,NaN,5.5455,79.13,1743.55,1743.55
4,#AAO,AAON,2017-03-31,Q,2017-05-04,0.1267,0.1267,0.000,NaN,USD,NaN,4.8240,78.94,1860.47,1860.47
5,#AAO,AAON,2017-06-30,Q,2017-08-03,0.1733,0.1733,0.000,NaN,USD,NaN,6.3349,78.94,1939.42,1939.42
6,#AAO,AAON,2017-09-30,Q,2017-11-02,0.1867,0.1867,0.000,NaN,USD,NaN,6.3904,78.90,1813.51,1813.51
7,#AAO,AAON,2017-12-31,Q,2018-02-27,0.1867,0.2000,0.000,NaN,USD,NaN,6.3661,78.70,1925.59,1925.59
8,#AAO,AAON,2018-03-31,Q,2018-05-03,0.0533,0.0533,0.000,NaN,USD,NaN,1.7779,78.64,2044.77,2044.77
9,#AAO,AAON,2018-06-30,Q,2018-08-02,0.1467,0.1467,0.000,NaN,USD,NaN,4.9032,78.54,1741.04,1741.04


In [19]:
merged_valid_tickers, merged_valid_tickers_count, merged_invalid_lens = valid_tickers(zacks_df, same_merge_date_tickers)
print(merged_valid_tickers_count)

1555


In [58]:
mktv_df.head(30)

,m_ticker,ticker,per_end_date,per_type,mkt_val
959,#AAO,AAON,2016-03-31,Q,1485.93
958,#AAO,AAON,2016-06-30,Q,1458.66
957,#AAO,AAON,2016-09-30,Q,1525.30
956,#AAO,AAON,2016-12-31,Q,1743.55
955,#AAO,AAON,2017-03-31,Q,1860.47
954,#AAO,AAON,2017-06-30,Q,1939.42
953,#AAO,AAON,2017-09-30,Q,1813.51
952,#AAO,AAON,2017-12-31,Q,1925.59
951,#AAO,AAON,2018-03-31,Q,2044.77
950,#AAO,AAON,2018-06-30,Q,1741.04


In [182]:
print(len(ratio_df['m_ticker'].unique()))
print(len(fundamental_df['m_ticker'].unique()))
print(len(shares_out_df['m_ticker'].unique()))
print(len(mktv_df['m_ticker'].unique()))
print(len(description_unique_ticker))

5173
5173
5173
5173
6122
